# **Transfer Learning and Fine Tuning using Resnet18 Architecture**


Sekarang kita akan menyiapkan *environment* untuk melakukan *finetuning* arsitektur Resnet18 menggunakan Timm. Kita mengimpor beberapa pustaka yang akan kita gunakan, termasuk [torchvision](https://pytorch.org/vision/stable/index.html) dan pustaka pendukung lainnya. Paket torchvision terdiri dari *dataset* populer, arsitektur model, dan transformasi gambar umum untuk *computer vision*.

In [2]:
!pip install timm torch torchvision

In [3]:
import torch #untuk membangun dan melatih model deep learning.
import torch.nn as nn #untuk membangun neural network.
import torch.optim as optim #untuk melatih model deep learning.
from torch.utils.data import DataLoader #untuk membuat iterable dari dataset.
from torchvision import datasets, transforms #Modul ini berisi dataset populer, arsitektur model, dan transformasi gambar umum untuk computer vision.
import timm #Merupakan library yang menyediakan koleksi model dan fungsi yang telah dilatih sebelumnya (pretrained) untuk computer vision.

## Data preparation and augmentation
Di sini, kita memfokuskan pada optimalisasi *dataset* untuk keperluan pelatihan dan validasi. Untuk mencapai hal tersebut, kita menggunakan *module* `transforms` dari PyTorch untuk menerapkan transformasi yang penting. Pada *training set*, kita menggunakan teknik [*resized transformation*](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#resize) dan [*Autoaugment transformation*](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#autoaugment) guna meningkatkan keragaman *dataset*, yang pada akhirnya membantu model untuk melakukan *generalization* dengan lebih baik. Selain itu, kita juga menerapkan **normalization** pada nilai piksel untuk *training set* dan *validation set* agar pemrosesan *neural network* menjadi lebih efisien. Langkah persiapan data yang komprehensif ini menjadi landasan kuat untuk proses pelatihan dan evaluasi model selanjutnya.

In [4]:
# Define data transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Sekarang kita akan memuat *dataset* untuk keperluan *training*.  
Dalam tutorial ini, kita akan menggunakan *CIFAR10*, sebuah *dataset* terkenal yang berisi gambar dari 10 kelas berbeda. *CIFAR-10* terdiri dari 60.000 gambar berukuran 32x32 berwarna, dengan 6.000 gambar per kelas. Tujuan utama kita adalah melatih model yang dapat melakukan *classification* pada gambar-gambar tersebut dengan akurat. Informasi lebih lanjut mengenai *dataset* ini akan dijelaskan di bagian selanjutnya.

**Catatan**: Anda dapat mengunduh *dataset* ini dari tautan berikut [link](https://www.cs.toronto.edu/~kriz/cifar.html). Data juga tersedia di [Hugging Face Datasets](https://huggingface.co/datasets/cifar10). Kita akan menggunakan *torchvision* dan *torch.utils.data* untuk memuat data. Anda dapat mempelajari lebih lanjut tentang pemuatan gambar pada [tutorial ini](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

In [5]:
# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:05<00:00, 29.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Kode pemrosesan *data sample* bisa dengan cepat menjadi sulit dikelola. Kita ingin meningkatkan keterbacaan dan modularitas dengan memisahkan kode untuk *dataset* dari kode untuk melatih model kita. Dalam hal ini, Python menyediakan dua *data primitives* melalui *torch.utils.data.DataLoader* dan *torch.utils.data*, yang memungkinkan kita memanfaatkan *dataset* kita sendiri maupun *dataset* yang sudah ada. Pada *dataset*, disimpan *sample* serta *label*-nya, dan untuk memudahkan akses terhadap data tersebut, *DataLoader* membungkus dataset dalam bentuk *iterable*.

In [6]:
# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Jadi, di dalam *dataset* ini terdapat 10.000 *test samples* dan 50.000 *training samples*. Selain itu, setiap kelas pada *training dataset* memiliki 6.000 gambar, sementara setiap kelas pada *validation dataset* memiliki 1.000 gambar.

## Fine-tuning the model

Kita sekarang akan menginisialisasi model kita, yaitu ResNet18. Perhatikan bahwa baris kode `model.fc = nn.Linear(model.fc.in_features, 10)` menggantikan lapisan *fully connected* (asli) terakhir dari model ResNet18 dengan lapisan baru yang memiliki 10 kelas. Hal ini menunjukkan bahwa saat model dilatih, model tersebut akan di-*fine-tune* untuk tugas baru dengan 10 kelas dari CIFAR10. Semua lapisan lainnya akan tetap “dibekukan” (*frozen*).

In [7]:
# Define the model
model = timm.create_model('resnet18', pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Kita mendefinisikan *loss function* untuk menghitung kesalahan prediksi. Pada contoh ini, kita akan menggunakan **Cross Entropy Loss** sebagai fungsi *loss* dan **Stochastic Gradient Descent (SGD)** sebagai *optimizer*-nya.

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


Sekarang kita akan melakukan *fine-tuning* model pada *dataset* CIFAR10.

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

Terakhir, kita akan memulai *loop* pelatihan dan evaluasi. Dalam tutorial ini, kita melatih data selama 30 *epoch*. Prosesnya akan melakukan iterasi pada *training dataset*, menghitung *loss* dan melakukan *backpropagation* untuk memperbarui parameter model. Setelah setiap *epoch*, model dievaluasi pada *validation dataset*, menghitung akurasi, lalu mencetak nomor *epoch*, nilai *loss*, dan *accuracy*. Tujuan akhirnya adalah meningkatkan kinerja model seiring dengan berjalannya proses pelatihan.

In [10]:
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')


Epoch 1/30, Loss: 1.9216, Accuracy: 0.4292
Epoch 2/30, Loss: 1.6193, Accuracy: 0.4950
Epoch 3/30, Loss: 1.5909, Accuracy: 0.5256
Epoch 4/30, Loss: 1.0846, Accuracy: 0.5373
Epoch 5/30, Loss: 1.0755, Accuracy: 0.5697
Epoch 6/30, Loss: 1.3195, Accuracy: 0.5697
Epoch 7/30, Loss: 1.3334, Accuracy: 0.5900
Epoch 8/30, Loss: 1.8448, Accuracy: 0.5961
Epoch 9/30, Loss: 1.3499, Accuracy: 0.6168
Epoch 10/30, Loss: 2.2109, Accuracy: 0.6200
Epoch 11/30, Loss: 1.1441, Accuracy: 0.6224
Epoch 12/30, Loss: 0.8558, Accuracy: 0.6324
Epoch 13/30, Loss: 1.7220, Accuracy: 0.6351
Epoch 14/30, Loss: 0.5184, Accuracy: 0.6447
Epoch 15/30, Loss: 1.1889, Accuracy: 0.6554
Epoch 16/30, Loss: 0.6687, Accuracy: 0.6546
Epoch 17/30, Loss: 0.8465, Accuracy: 0.6651
Epoch 18/30, Loss: 0.7728, Accuracy: 0.6637
Epoch 19/30, Loss: 1.0088, Accuracy: 0.6747
Epoch 20/30, Loss: 0.9711, Accuracy: 0.6709
Epoch 21/30, Loss: 1.5154, Accuracy: 0.6741
Epoch 22/30, Loss: 0.6587, Accuracy: 0.6744
Epoch 23/30, Loss: 0.5036, Accuracy: 0.68

In [11]:
# Save the trained model
torch.save(model.state_dict(), 'resnet18_cifar10.pth')

Let's make predictions on custom images using the trained model, and see the predicted class labels displayed next to the images.

In [12]:
from PIL import Image
import torchvision.transforms as transforms

# Load the trained model
model = timm.create_model('resnet18')
model.fc = nn.Linear(model.fc.in_features, 10)  # Change the last fully connected layer for CIFAR-10
model.load_state_dict(torch.load('resnet18_cifar10.pth'))
model.eval()

<ipython-input-12-fe961eaa6178>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('resnet18_cifar10.pth'))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [13]:
image_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [29]:
from PIL import Image
import requests
from io import BytesIO
url = "https://madbarn.com/wp-content/uploads/2023/12/List-of-Gaited-Horse-Breeds.jpg"
response = requests.get(url)
image = Image.open(BytesIO(response.content))

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x79c4288aff10>

In [31]:
# image_path = '/content/images.jpeg'
# image = Image.open(image_path)
input_image = image_transform(image).unsqueeze(0)  # Add batch dimension

NameError: name 'image' is not defined

In [32]:
# Make prediction
import matplotlib.pyplot as plt
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]
plt.imshow(image)
with torch.no_grad():
    model_output = model(input_image)
    _, predicted_class = torch.max(model_output, 1)

predicted_class_index = predicted_class.item()
predicted_class_name = class_names[predicted_class_index]

print(f'Predicted class: {predicted_class_index} - {predicted_class_name}')

NameError: name 'image' is not defined